In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%%capture
!pip install wandb

In [3]:
import wandb
from wandb.keras import WandbCallback

wandb.login()

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [4]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import Normalizer
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [5]:
df_train = pd.read_csv('/content/drive/MyDrive/Redes Neurais/Github/Redes-Neurais/Titanic/train_tratado.csv')

In [6]:
train = df_train.drop(columns = ['Transported'])
y = df_train['Transported']

### Normalizar

transformer = Normalizer().fit(train) 
transformer
train=transformer.transform(train)


#Splitar em treinamento e teste

x_train, x_test, y_train, y_test = train_test_split(train, y, test_size=0.3)
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(4624, 27)
(4624,)
(1982, 27)
(1982,)


####Tunar hiperparametro

In [31]:
sweep_config = {
  'method': 'bayes', 
  'metric': {
      'name': 'loss',
      'goal': 'minimize'
  },
 'early_terminate':{'type': 'hyperband','min_iter': 5},
  'parameters': {
      #'batch_size': {'values': [100]}
      'learning_rate':{'values': [0.0001, 0.001]}
      ,'dropout_1':{'values':[0.5]}
      #,'dropout_2':{'values':[0.3,0.4,0.5,0.8,0.95]}
      #,'l1_size':{'values':[16,32,50,64,128]}
      #,'l2_size':{'values':[16,32,50,64,128]}
      #,'fc1_size':{'values':[2,4,8,16,32]}
      #,'fc2_size':{'values':[2,4,8,16,32]}
      #,'fc3_size':{'values':[2,4,8,16,32]}
      #,'fc4_size':{'values':[2,4,8,16,32]}
      ,'fc_layer_size': {'values':[32,64,128,256,512,1024]}
      #,'optimizer': {'values': ['adam','adamax']}

  }
}

In [33]:
def sweep_train(config_defaults=None):
    # Set default values
    config_defaults = {
        "batch_size": 64,
        "learning_rate": 0.01
    }
    # Initialize wandb with a sample project name
    wandb.init()  # this gets over-written in the Sweep

    # Specify the other hyperparameters to the configuration, if any
    '''    wandb.config.epochs = 2
    wandb.config.log_step = 20
    wandb.config.val_log_step = 50
    wandb.config.architecture_name = "MLP"
    wandb.config.dataset_name = "MNIST"'''


    model = Sequential() 
    model.add(Dense(wandb.config.fc_layer_size, activation='relu', input_dim=train.shape[1]))
    model.add(Dropout(wandb.config.dropout_1))
    model.add(Dense(wandb.config.fc_layer_size, activation='relu'))
    model.add(Dropout(wandb.config.dropout_1))
    model.add(Dense(wandb.config.fc_layer_size, activation='relu'))
    model.add(Dropout(wandb.config.dropout_1))
    model.add(Dense(wandb.config.fc_layer_size, activation='relu'))
    model.add(Dropout(wandb.config.dropout_1))
    model.add(Dense(1, activation='sigmoid')) 

    model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=wandb.config.learning_rate), metrics=['accuracy']) 

    

    model.fit(x_train, y_train, epochs=150, validation_data=(x_test, y_test), callbacks=[WandbCallback()])

In [34]:
sweep_id = wandb.sweep(sweep_config, project="teste_loss")

Create sweep with ID: jfx0lsep
Sweep URL: https://wandb.ai/lucasalmada/teste_loss/sweeps/jfx0lsep


In [ ]:
wandb.agent(sweep_id, function=sweep_train,count=15)

wandb: Agent Starting Run: bs3rt1i6 with config:
wandb: 	dropout_1: 0.5
wandb: 	fc_layer_size: 256
wandb: 	learning_rate: 0.001
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Epoch 1/150
140/145 [===========================>..] - ETA: 0s - loss: 0.5784 - accuracy: 0.7123

wandb: Adding directory to artifact (/content/wandb/run-20221206_025830-bs3rt1i6/files/model-best)... Done. 0.0s


145/145 [==============================] - 3s 18ms/step - loss: 0.5763 - accuracy: 0.7143 - val_loss: 0.4785 - val_accuracy: 0.7972
Epoch 2/150
139/145 [===========================>..] - ETA: 0s - loss: 0.5097 - accuracy: 0.7851

wandb: Adding directory to artifact (/content/wandb/run-20221206_025830-bs3rt1i6/files/model-best)... Done. 0.0s


...

145/145 [==============================] - 10s 71ms/step - loss: 0.4660 - accuracy: 0.7913 - val_loss: 0.4345 - val_accuracy: 0.8098
Epoch 16/150
145/145 [==============================] - 9s 62ms/step - loss: 0.4621 - accuracy: 0.7937 - val_loss: 0.4392 - val_accuracy: 0.8022
Epoch 17/150
144/145 [============================>.] - ETA: 0s - loss: 0.4645 - accuracy: 0.7947

wandb: Adding directory to artifact (/content/wandb/run-20221206_042206-zx1ved9m/files/model-best)... Done. 0.3s


145/145 [==============================] - 11s 75ms/step - loss: 0.4654 - accuracy: 0.7943 - val_loss: 0.4330 - val_accuracy: 0.8007
Epoch 18/150
145/145 [==============================] - 9s 62ms/step - loss: 0.4598 - accuracy: 0.7935 - val_loss: 0.4339 - val_accuracy: 0.8103
Epoch 19/150
145/145 [==============================] - 9s 61ms/step - loss: 0.4649 - accuracy: 0.7950 - val_loss: 0.4506 - val_accuracy: 0.7987
Epoch 20/150
145/145 [==============================] - 9s 59ms/step - loss: 0.4613 - accuracy: 0.7930 - val_loss: 0.4402 - val_accuracy: 0.8037
Epoch 21/150
145/145 [==============================] - 8s 59ms/step - loss: 0.4557 - accuracy: 0.7930 - val_loss: 0.4400 - val_accuracy: 0.8037
Epoch 22/150
145/145 [==============================] - 8s 58ms/step - loss: 0.4555 - accuracy: 0.7933 - val_loss: 0.4375 - val_accuracy: 0.8037
Epoch 23/150
145/145 [==============================] - 9s 59ms/step - loss: 0.4602 - accuracy: 0.7920 - val_loss: 0.4534 - val_accuracy: 0.80

wandb: Adding directory to artifact (/content/wandb/run-20221206_042206-zx1ved9m/files/model-best)... Done. 0.3s


145/145 [==============================] - 10s 70ms/step - loss: 0.4533 - accuracy: 0.7956 - val_loss: 0.4212 - val_accuracy: 0.8063
Epoch 25/150
145/145 [==============================] - 9s 59ms/step - loss: 0.4558 - accuracy: 0.7924 - val_loss: 0.4306 - val_accuracy: 0.8052
Epoch 26/150
145/145 [==============================] - 8s 59ms/step - loss: 0.4481 - accuracy: 0.7933 - val_loss: 0.4339 - val_accuracy: 0.8042
Epoch 27/150
145/145 [==============================] - 9s 59ms/step - loss: 0.4551 - accuracy: 0.7928 - val_loss: 0.4247 - val_accuracy: 0.8047
Epoch 28/150
145/145 [==============================] - 8s 59ms/step - loss: 0.4470 - accuracy: 0.7943 - val_loss: 0.4345 - val_accuracy: 0.8098
Epoch 29/150
145/145 [==============================] - 8s 58ms/step - loss: 0.4417 - accuracy: 0.7967 - val_loss: 0.4333 - val_accuracy: 0.8052
Epoch 30/150
145/145 [==============================] - 9s 59ms/step - loss: 0.4550 - accuracy: 0.7948 - val_loss: 0.4266 - val_accuracy: 0.80

In [ ]:
wandb.agent('ig4a3led', function=sweep_train, count=10)

wandb: Agent Starting Run: ydphoee3 with config:
wandb: 	architecture_name: MLP
wandb: 	batch_size: 38
wandb: 	dataset_name: MNIST
wandb: 	epochs: 3
wandb: 	learning_rate: 0.018443613969138465
wandb: 	log_step: 26
wandb: 	val_log_step: 99
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Epoch 1/50
163/166 [============================>.] - ETA: 0s - loss: 0.4056 - accuracy: 0.8014

wandb: Adding directory to artifact (/content/wandb/run-20221130_022204-ydphoee3/files/model-best)... Done. 0.1s


166/166 [==============================] - 3s 16ms/step - loss: 0.4058 - accuracy: 0.8019 - val_loss: 0.4872 - val_accuracy: 0.8064
Epoch 2/50
165/166 [============================>.] - ETA: 0s - loss: 0.4101 - accuracy: 0.8068

wandb: Adding directory to artifact (/content/wandb/run-20221130_022204-ydphoee3/files/model-best)... Done. 0.1s


...

wandb: Adding directory to artifact (/content/wandb/run-20221130_023327-cajla11c/files/model-best)... Done. 0.1s


166/166 [==============================] - 2s 12ms/step - loss: 0.1840 - accuracy: 0.9129 - val_loss: 1.1585 - val_accuracy: 0.7640
Epoch 7/50
158/166 [===========================>..] - ETA: 0s - loss: 0.1651 - accuracy: 0.9159

wandb: Adding directory to artifact (/content/wandb/run-20221130_023327-cajla11c/files/model-best)... Done. 0.1s


166/166 [==============================] - 3s 16ms/step - loss: 0.1686 - accuracy: 0.9152 - val_loss: 1.0798 - val_accuracy: 0.7572
Epoch 8/50
166/166 [==============================] - 1s 7ms/step - loss: 0.1564 - accuracy: 0.9228 - val_loss: 1.0969 - val_accuracy: 0.7579
Epoch 9/50
166/166 [==============================] - 1s 5ms/step - loss: 0.1536 - accuracy: 0.9234 - val_loss: 1.2471 - val_accuracy: 0.7519
Epoch 10/50
166/166 [==============================] - 1s 5ms/step - loss: 0.1493 - accuracy: 0.9232 - val_loss: 1.1550 - val_accuracy: 0.7617
Epoch 11/50
166/166 [==============================] - 1s 7ms/step - loss: 0.1611 - accuracy: 0.9190 - val_loss: 1.2671 - val_accuracy: 0.7436
Epoch 12/50
166/166 [==============================] - 1s 5ms/step - loss: 0.1574 - accuracy: 0.9235 - val_loss: 1.1557 - val_accuracy: 0.7572
Epoch 13/50
166/166 [==============================] - 1s 6ms/step - loss: 0.1482 - accuracy: 0.9264 - val_loss: 1.2145 - val_accuracy: 0.7761
Epoch 14/50


accuracy,▅▅▅▁▁▂▅▅▄▆▇▆▄▄▄▄▅▆▆▅▄▃▆▇▆▅▇█▅▄▅▇▃▅▇██▇▇▅
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
loss,▄▄▄▇█▆▄▄▅▄▃▂▅▅▆▆▄▃▃▃▅▇▄▂▃▃▂▂▄▅▄█▇▄▂▁▂▂▃▅
val_accuracy,▆▇▇▆▆▅▅▄▃▅██▆▆▁▅▇▇▅▆▇▅▆▆▆▅▇▅▆▆▄▅▃▅▅▅▄▃▄▄
val_loss,▃▃▃▃▂▁▁▄▄▂▃▆▄▅▇▃▃▃▅▇▃▂▄▄▄▆▆█▆▇▅█▃▄▄▆█▇▅▅
accuracy,0.92316
best_epoch,6
best_val_loss,1.07981
epoch,49
loss,0.16343
val_accuracy,0.75113


In [ ]:
sweep_id

'wqux4o69'